# Практическая работа №3: Обработка выборочных данных. Нахождение интервальных оценок параметров распределения. Проверка статистической гипотезы о нормальном распределении.
Выполнили студенты гр. 0381 Котов Дмитрий и Михайлов Виктор. Вариант №8

## Цель работы

Получение практических навыков вычисления интервальных статистических оценок параметров распределения выборочных данных и проверки «справедливости» статистических гипотез.

## Основные теоретические положения

Доверительным называют интервал, который с заданной надежностью 𝛾 покрывает заданный параметр.

Интервальной оценкой математического ожидания при неизвестном среднем квадратическом отклонении 𝜎 генеральной совокупности служит доверительный интервал:

$\bar{x_в} - \frac{s}{\sqrt{n}}t_\gamma \leqslant \alpha \leqslant \bar{x_в} + \frac{s}{\sqrt{n}}t_\gamma$

$\bar{x_в}$ – статистическая оценка математического ожидания

$s$ – исправленное СКВО

$n$ – объём выборки

$t_\gamma$ – из таблицы

Доверительный интервал для оценки СКВО:

$s(1-q)\leqslant \sigma \leqslant s(1+q)$

$s$ – исправленное СКВО

$q$ – из таблицы

Критерий Пирсона, или критерий $\chi^2$(Хи-квадрат), применяют для проверки гипотезы о соответствии эмпирического распределения предполагаемому теоретическому распределению.

Метод позволяет оценить статистическую значимость различий двух или нескольких относительных показателей.

Теоретические частоты вычисляются по формуле:

${n_i}' = p_i N$

$p_i = \Phi(z_{i+1}) - \Phi(z_i)$

где $\Phi(z_i)$ – функция Лапласа

Если $\chi^2_{наб} \leqslant \chi^2_{крит}$ – гипотеза принимается, иначе $(\chi^2_{наб} > \chi^2_{крит})$ – гипотезу отвергают.

## Постановка задачи

Для заданной надежности определить (на основании выборочных данных и результатов выполнения практической работы №2) границы доверительных интервалов для математического ожидания и среднеквадратичного отклонения случайной величины. Проверить гипотезу о нормальном распределении исследуемой случайной величины с помощью критерия Пирсона $\chi^2$
. Дать содержательную интерпретацию полученным результатам.

## Выполнение работы

### Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.stats import skew
from scipy.stats import kurtosis

### Восстановим результаты практической работы №1 (выборочные данные, которые необходимо обработать)

In [ ]:
data = pd.read_csv("/content/sample.csv")
data = data.sample(n=118, random_state=118)
data = data.sort_index()
data = data.reset_index(drop=True)
data.to_csv("/content/выборка.csv", index=False)

df = pd.read_csv("/content/выборка.csv")

df_nu = df.drop("E", axis=1)

df_nu = df_nu.sort_values(by=["nu"], ignore_index = True)

x = df_nu['nu']
var_series = pd.DataFrame({'nu': x.value_counts().sort_index().index})
var_series # Вариационный ряд

# Интервальный ряд
n=118
k = int(np.floor(1+3.31*np.log10(n))) # количество интервалов
h = int(np.ceil((max(x)-min(x))/k)) # ширина интервала
interval_bounds = [] # границы интервалов
x_min = min(var_series['nu'])
interval_bounds.append(x_min)
for i in range(1, k):
  interval_bounds.append(x_min+h*i)
interval_bounds.append(x_min+k*h)

counts = x.value_counts().sort_index().values # абсолютные частоты
rel_counts = x.value_counts(normalize=True).sort_index().values # относительные частоты

interval_series = pd.concat([var_series, pd.DataFrame({'n_i': counts, 'p_i': rel_counts})], axis=1)
interval_series["interval_i-i+1"] = pd.cut(interval_series["nu"], bins=interval_bounds, right=False)
interval_series = interval_series.groupby(["interval_i-i+1"])[["n_i", "p_i"]].apply(sum).reset_index()
interval_series["avg_i"] = np.array([np.mean([interval_bounds[i], interval_bounds[i+1]], axis=0) for i in range(k)])
interval_series = interval_series[["interval_i-i+1", "avg_i", "n_i", "p_i"]]

# Добавим накопленные частоты
n_cum = [] # накопленные абсолютные частоты
p_cum = [] # накопленные относительные частоты

n_abs, p_abs = interval_series['n_i'], interval_series['p_i']

for i in range(len(n_abs)):
  n_cum.append(sum(n_abs[:i+1]))
  p_cum.append(sum(p_abs[:i+1]))

interval_series['n_cum_i'] = n_cum
interval_series['p_cum_i'] = p_cum

### 1. Вычислить точность и доверительный интервал для математического ожидания при неизвестном среднеквадратичном отклонении при заданном объёме выборки для доверительной точности $γ∈[0.95;0.99]$

Доверительный интервал для математического ожидания:
$(\overline{x}_в - t_γs/\sqrt{N} ; \overline{x}_в + t_γs/\sqrt{N})$

Пусть $γ=0.95$. Тогда по таблице распределения Стьюдента определяем $t_γ=1.984$

Таким образом вычислим доверительный интервал с надёжностью $γ=0.95$:

In [ ]:
t_gamma = 1.984
x_mean1 = sum(interval_series["avg_i"]*interval_series["n_i"])/len(x)  # выбочроное среднее
s = np.sqrt(np.var(x, ddof = 1))  # среднеквадратическое отклонение исправленной дисперсии
bound_left = x_mean1-(t_gamma*s)/np.sqrt(len(x))
bound_right = x_mean1+(t_gamma*s)/np.sqrt(len(x))
print(bound_left, bound_right)

442.9045243954929 464.4853061129817


Получили следующий доверительный интервал: $(442.9; 464.49)$

Вычислим точность $(δ= t_γs/\sqrt{N})$ покрытия истинного значения мат. ожидания данным доверительным интервалом:

In [ ]:
delta = (t_gamma*s)/np.sqrt(len(x))
delta

10.790390858744377

Аналогичным образом можно вычислить точность и доверительный интервал для мат. ожидания при $γ$ равном, например, $0.99$ и $t_γ=2.627$:

- $(439.4; 467.98)$ — доверительный интервал
- $14.29$ — точность

**Выводы**:
- с веротностью 95% значение мат. ожидания покрывается интервалом $(442.9; 464.49)$
- с веротностью 99% значение мат. ожидания покрывается интервалом $(439.4; 467.98)$
- чем больше значение надёжности, тем менее точен доверительный интервал

###2. Для вычисления границ доверительного интервала для среднеквадратичного отклонения определить значение $q$ при заданных $γ$ и $n$. Построить доверительные интервалы.



$q = 0.143$ из таблицы для $\gamma = 0.95$ и $N = 100$

In [ ]:
q = 0.143
print("(",s*(1-q),",", s*(1+q),")")

( 50.631096646705686 , 67.5278220153846 )


$q = 0.198$ из таблицы для $\gamma = 0.99$ и $N = 100$

In [ ]:
q = 0.198
print("(",s*(1-q),",", s*(1+q),")")

( 47.381726383498204 , 70.77719227859208 )


**Выводы**:
- с веротностью 95% значение СКО покрывается интервалом $(50.63; 67.53)$
- с веротностью 99% значение СКО покрывается интервалом $(47.38; 70.77)$
- чем больше значение надёжности, тем менее точен доверительный интервал

###3. Проверить гипотезу о нормальности заданного распределения с помощью критерия $\chi^2$(Пирсона). Для этого необходимо найти теоретические частоты и вычислить наблюдаемое значение критерия.

Пересчитаем границы интервалов:

In [ ]:
import math
z = []
for bound in interval_bounds:
  z.append((bound - x_mean1)/s)
z[0] = -math.inf
z[-1] = math.inf
z

[-inf,
 -1.4335763430003448,
 -0.7903747898671102,
 -0.14717323673387567,
 0.49602831639935885,
 1.1392298695325933,
 1.7824314226658278,
 inf]

Вычислим через функцию Лапласа вероятности попадания в каждый интервал:

In [ ]:
from scipy import stats
p = []
for i in range(k):
  p.append(stats.norm.cdf(z[i+1]) - stats.norm.cdf(z[i]))
p

[0.07584659924851812,
 0.13880786106008103,
 0.22684318196012127,
 0.24856514128933732,
 0.18263357127658453,
 0.08996419261294075,
 0.037339452552416996]

Получим теоретические частоты:

In [ ]:
n_shtrix = []
for i in range(k):
  n_shtrix.append(p[i]*n)
n_shtrix

[8.949898711325138,
 16.37932760508956,
 26.76749547129431,
 29.330686672141805,
 21.550761410636973,
 10.61577472832701,
 4.406055401185205]

Вычислим значение критерия Пирсона:

In [ ]:
sum(((interval_series['n_i']-n_shtrix)**2)/(n_shtrix))

3.3638841182649464

Критическое значение критерия Пирсона = 9.5

Т.к. 3.36 < 9.5, то гипотеза о нормальности распределения принимается с уровнем значимости 0.05 и количеством степеней свободы 7-3=4

| $i$      | $(x_{i-1},x_i]$ | $n_i$ | $p_i$                | $n_i'$             | $(n_i-n_i')^2$        | $\frac{(n_i-n_i')^2}{n_i'}$ | $n_i^2$ | $\frac{n_i^2}{n_i'}$ |
|----------|-----------------|-------|----------------------|--------------------|-----------------------|-----------------------------|---------|----------------------|
| 1        | (331,369)       | 9.0   | 0.07584659924851812  | 8.949898711325138  | 0.0025101391268818074 | 0.0002804656463548012       | 81.0    | 9.050381754321217    |
| 2        | (369,407)       | 17.0  | 0.13880786106008103  | 16.37932760508956  | 0.38523422180386035   | 0.02351953823087074         | 289.0   | 17.64419193314131    |
| 3        | (407,445)       | 29.0  | 0.22684318196012127  | 26.76749547129431  | 4.984076470691413     | 0.18619883492784667         | 841.0   | 31.418703363633536   |
| 4        | (445,483)       | 29.0  | 0.24856514128933732  | 29.330686672141805 | 0.10935367513222187   | 0.0037283025915682243       | 841.0   | 28.673041630449763   |
|          | (483,521)       | 15.0  | 0.18263357127658453  | 21.550761410636973 | 42.912475059090504    | 1.9912277919753623          | 225.0   | 10.44046638133839    |
|          | (521,559)       | 14.0  | 0.08996419261294075  | 10.61577472832701  | 11.452980689430127    | 1.0788643299738763          | 196.0   | 18.463089601646868   |
|          | (559,597)       | 5.0   | 0.037339452552416996 | 4.406055401185205  | 0.3527701864612674    | 0.08006485491906754         | 25.0    | 5.674009453733862    |
| $\Sigma$ |                 | 118.0 | 1.0                  | 118.0              | 60.2                  | 3.36                        | 2498.0  | 121.36               |

###4. Доказать, что

$\displaystyle \chi^2_{набл} = \sum^k_{i=1} \frac{n^2_i}{{n_i}'}-n$

Проконтролировать корректность вычисления $\chi^2_{набл}$.

Согласно критерию Пирсона: $$χ^2_{набл} = \sum^k_{i=1} \frac{(n_i - n_i')^2}{n_i'} $$

$\displaystyle \sum^k_{i=1} \frac{(n_i - n_i')^2}{n_i'}  = \sum^k_{i=1} \frac{n_i^2 - 2 n_i n_i' + n_i'^2}{n_i'}  = \sum^k_{i=1}\frac{n_i^2}{n_i'} - 2\sum^k_{i=1} n_i + \sum^k_{i=1} n_i'$

Так как $\displaystyle \sum^k_{i=1} n_i = \sum^k_{i=1} n_i' = n$, получаем:

$$\chi^2_{\textit{набл}} = \sum_{i = 1}^k\frac{n^2_i}{n'_i} - n.$$

In [ ]:
chi2_nabl = sum(((interval_series['n_i']-n_shtrix)**2)/(n_shtrix))

print(round(chi2_nabl,2),"=",round(121.36 - n,2))


3.36 = 3.36


$χ^2_{набл} = 3.36$ значения совпадают

###5. По заданному уровню значимости $\alpha = 0.05$ и числу степеней свободы $df$ найти критическую точку $\chi^2_{крит}$ и сравнить с наблюдаемым значением.

В пункте 3 уже было вычислено $\chi^2_{крит}$ и оно оказалось больше $\chi^2_{набл}$, поэтому нулевая гипотеза была принята


## Выводы

В ходе выполнения данной лабораторной работы были выполнены вычисления интервальных статистических оценок параметров распределения выборочных данных и проверки «справедливости» статистических гипотез. Построение доверительных интервалов показало, что повышение надёжности приводит к снижению точности оценки, однако точность может быть повышена также с помощью увеличения размера выборки. Была выдвинута гипотеза о нормальности распределения выборочных данных. Проверка этой гипотезы по критерию Пирсона дала основания принять её с выбранным уровнем значимости. Доказана эквивалентность формул для вычисления наблюдённого значения критерия Пирсона.